# Transfer Learning

## 1.There is a trained model we want to re-train some of the layers on our custom datased.
## 2.Training a pretrained model completely on custom dataset is transfer learning


1. Large and Different Dataset >> Fine-Tuning
2. Large and Similar Dataset >> Fine-Tuning
3. Small and Different Dataset >> Fine-Tuning
4. Small and Similar Dataset >> Transfer-Learning

In [2]:
%%html
<img src='https://www.topbots.com/wp-content/uploads/2019/12/cover_transfer_learning_1600px_web-1280x640.jpg', width=600, height=300>

In [19]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!pip install wget

In [21]:
import wget
wget.download('https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip', './cats_and_dogs_filtered.zip')

  1% [.                                                                       ]   974848 / 68606236

100% [........................................................................] 68606236 / 68606236

'./cats_and_dogs_filtered.zip'

In [1]:
!pip install tensorflow-gpu==2.1.0

In [2]:
import zipfile

In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
%matplotlib inline

In [5]:
print(tf.__version__)

2.1.0


In [6]:
dir(ImageDataGenerator)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_keras_api_names',
 '_keras_api_names_v1',
 'apply_transform',
 'fit',
 'flow',
 'flow_from_dataframe',
 'flow_from_directory',
 'get_random_transform',
 'random_transform',
 'standardize']

In [7]:
file_extract_object = zipfile.ZipFile(file='./cats_and_dogs_filtered.zip', mode='r')

In [8]:
file_extract_object.extractall()

In [9]:
file_extract_object.close()

In [7]:
dataset_dir = './cats_and_dogs_filtered/'
train_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'validation')
print(train_dir)
print(validation_dir)


./cats_and_dogs_filtered/train
./cats_and_dogs_filtered/validation


### MobileNetV2

In [49]:
IMAGE_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMAGE_SHAPE, include_top=False, weights='imagenet')

In [50]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

block_4_expand_relu (ReLU)      (None, 16, 16, 192)  0           block_4_expand_BN[0][0]          
__________________________________________________________________________________________________
block_4_depthwise (DepthwiseCon (None, 16, 16, 192)  1728        block_4_expand_relu[0][0]        
__________________________________________________________________________________________________
block_4_depthwise_BN (BatchNorm (None, 16, 16, 192)  768         block_4_depthwise[0][0]          
__________________________________________________________________________________________________
block_4_depthwise_relu (ReLU)   (None, 16, 16, 192)  0           block_4_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_4_project (Conv2D)        (None, 16, 16, 32)   6144        block_4_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_4_pr

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, 8, 8, 64)     24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, 8, 8, 64)     256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, 8, 8, 64)     0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, 8, 8, 384)    24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

__________________________________________________________________________________________________
block_13_expand_BN (BatchNormal (None, 8, 8, 576)    2304        block_13_expand[0][0]            
__________________________________________________________________________________________________
block_13_expand_relu (ReLU)     (None, 8, 8, 576)    0           block_13_expand_BN[0][0]         
__________________________________________________________________________________________________
block_13_pad (ZeroPadding2D)    (None, 9, 9, 576)    0           block_13_expand_relu[0][0]       
__________________________________________________________________________________________________
block_13_depthwise (DepthwiseCo (None, 4, 4, 576)    5184        block_13_pad[0][0]               
__________________________________________________________________________________________________
block_13_depthwise_BN (BatchNor (None, 4, 4, 576)    2304        block_13_depthwise[0][0]         
__________

## Freezing the base model

In [51]:
base_model.trainable = False

## Defining custom head for our network

In [52]:
base_model.output

<tf.Tensor 'out_relu_1/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [53]:
base_model.input

<tf.Tensor 'input_2:0' shape=(None, 128, 128, 3) dtype=float32>

In [54]:
global_average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [55]:
global_average_pooling_layer

<tf.Tensor 'global_average_pooling2d_2/Identity:0' shape=(None, 1280) dtype=float32>

In [56]:
custom_layer1 = tf.keras.layers.Dense(units=128, activation='relu')(global_average_pooling_layer)
custom_layer2 = tf.keras.layers.Dense(units=64, activation='relu')(custom_layer1)

In [57]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(custom_layer2)

In [58]:
prediction_layer

<tf.Tensor 'dense_4/Identity:0' shape=(None, 1) dtype=float32>

In [59]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [60]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

block_4_expand_relu (ReLU)      (None, 16, 16, 192)  0           block_4_expand_BN[0][0]          
__________________________________________________________________________________________________
block_4_depthwise (DepthwiseCon (None, 16, 16, 192)  1728        block_4_expand_relu[0][0]        
__________________________________________________________________________________________________
block_4_depthwise_BN (BatchNorm (None, 16, 16, 192)  768         block_4_depthwise[0][0]          
__________________________________________________________________________________________________
block_4_depthwise_relu (ReLU)   (None, 16, 16, 192)  0           block_4_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_4_project (Conv2D)        (None, 16, 16, 32)   6144        block_4_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_4_pr

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, 8, 8, 64)     24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, 8, 8, 64)     256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, 8, 8, 64)     0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, 8, 8, 384)    24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

__________________________________________________________________________________________________
block_13_expand_BN (BatchNormal (None, 8, 8, 576)    2304        block_13_expand[0][0]            
__________________________________________________________________________________________________
block_13_expand_relu (ReLU)     (None, 8, 8, 576)    0           block_13_expand_BN[0][0]         
__________________________________________________________________________________________________
block_13_pad (ZeroPadding2D)    (None, 9, 9, 576)    0           block_13_expand_relu[0][0]       
__________________________________________________________________________________________________
block_13_depthwise (DepthwiseCo (None, 4, 4, 576)    5184        block_13_pad[0][0]               
__________________________________________________________________________________________________
block_13_depthwise_BN (BatchNor (None, 4, 4, 576)    2304        block_13_depthwise[0][0]         
__________

__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 1)            65          dense_3[0][0]                    
Total params: 2,430,273
Trainable params: 172,289
Non-trainable params: 2,257,984
__________________________________________________________________________________________________


## Compile the model

In [61]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [62]:
data_gen_train = ImageDataGenerator(rescale=255.)
data_gen_test = ImageDataGenerator(rescale=255.)

In [63]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128, 128), class_mode='binary')
validation_generator = data_gen_test.flow_from_directory(validation_dir, target_size = (128, 128), class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [64]:
model.fit_generator(train_generator, epochs=5,validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 63 steps, validate for 32 steps
Epoch 1/15
63/63 [==============================] - ETA: 2:44 - loss: 0.7855 - accuracy: 0.40 - ETA: 1:26 - loss: 0.7343 - accuracy: 0.48 - ETA: 1:00 - loss: 0.6808 - accuracy: 0.56 - ETA: 47s - loss: 0.6633 - accuracy: 0.5859 - ETA: 40s - loss: 0.6591 - accuracy: 0.593 - ETA: 34s - loss: 0.6495 - accuracy: 0.604 - ETA: 30s - loss: 0.6468 - accuracy: 0.601 - ETA: 27s - loss: 0.6404 - accuracy: 0.620 - ETA: 24s - loss: 0.6348 - accuracy: 0.621 - ETA: 22s - loss: 0.6327 - accuracy: 0.628 - ETA: 21s - loss: 0.6240 - accuracy: 0.639 - ETA: 20s - loss: 0.6181 - accuracy: 0.646 - ETA: 18s - loss: 0.6059 - accuracy: 0.665 - ETA: 17s - loss: 0.5989 - accuracy: 0.678 - ETA: 16s - loss: 0.5916 - accuracy: 0.694 - ETA: 16s - loss: 0.5857 - accuracy: 0.703 - ETA: 15s - loss: 0.5883 - accuracy: 0.695 - ETA: 14s - loss: 0.5809 - accuracy: 0.705 - ETA: 14s - loss: 0.5725 - accuracy: 0.716 - ETA: 13s - loss: 0.

63/63 [==============================] - ETA: 20s - loss: 0.1391 - accuracy: 0.937 - ETA: 16s - loss: 0.1790 - accuracy: 0.921 - ETA: 14s - loss: 0.1777 - accuracy: 0.947 - ETA: 13s - loss: 0.1529 - accuracy: 0.960 - ETA: 13s - loss: 0.1475 - accuracy: 0.956 - ETA: 12s - loss: 0.1509 - accuracy: 0.958 - ETA: 12s - loss: 0.1510 - accuracy: 0.959 - ETA: 11s - loss: 0.1549 - accuracy: 0.960 - ETA: 11s - loss: 0.1611 - accuracy: 0.954 - ETA: 11s - loss: 0.1615 - accuracy: 0.953 - ETA: 11s - loss: 0.1681 - accuracy: 0.946 - ETA: 10s - loss: 0.1680 - accuracy: 0.947 - ETA: 10s - loss: 0.1644 - accuracy: 0.947 - ETA: 10s - loss: 0.1701 - accuracy: 0.939 - ETA: 10s - loss: 0.1653 - accuracy: 0.941 - ETA: 9s - loss: 0.1691 - accuracy: 0.939 - ETA: 9s - loss: 0.1642 - accuracy: 0.94 - ETA: 9s - loss: 0.1728 - accuracy: 0.93 - ETA: 9s - loss: 0.1725 - accuracy: 0.93 - ETA: 8s - loss: 0.1680 - accuracy: 0.93 - ETA: 8s - loss: 0.1683 - accuracy: 0.93 - ETA: 8s - loss: 0.1674 - accuracy: 0.93 - ETA:

63/63 [==============================] - ETA: 18s - loss: 0.1058 - accuracy: 0.968 - ETA: 15s - loss: 0.1347 - accuracy: 0.953 - ETA: 13s - loss: 0.1040 - accuracy: 0.968 - ETA: 13s - loss: 0.1206 - accuracy: 0.968 - ETA: 12s - loss: 0.1503 - accuracy: 0.937 - ETA: 12s - loss: 0.1609 - accuracy: 0.932 - ETA: 11s - loss: 0.1536 - accuracy: 0.937 - ETA: 11s - loss: 0.1532 - accuracy: 0.941 - ETA: 11s - loss: 0.1456 - accuracy: 0.944 - ETA: 10s - loss: 0.1425 - accuracy: 0.943 - ETA: 10s - loss: 0.1340 - accuracy: 0.948 - ETA: 10s - loss: 0.1286 - accuracy: 0.953 - ETA: 10s - loss: 0.1231 - accuracy: 0.954 - ETA: 9s - loss: 0.1275 - accuracy: 0.955 - ETA: 9s - loss: 0.1226 - accuracy: 0.95 - ETA: 9s - loss: 0.1197 - accuracy: 0.95 - ETA: 9s - loss: 0.1184 - accuracy: 0.95 - ETA: 8s - loss: 0.1189 - accuracy: 0.96 - ETA: 8s - loss: 0.1178 - accuracy: 0.96 - ETA: 8s - loss: 0.1150 - accuracy: 0.96 - ETA: 8s - loss: 0.1144 - accuracy: 0.96 - ETA: 8s - loss: 0.1148 - accuracy: 0.96 - ETA: 7s 

63/63 [==============================] - ETA: 19s - loss: 0.0774 - accuracy: 1.000 - ETA: 15s - loss: 0.0683 - accuracy: 1.000 - ETA: 14s - loss: 0.0576 - accuracy: 1.000 - ETA: 13s - loss: 0.0547 - accuracy: 1.000 - ETA: 12s - loss: 0.0522 - accuracy: 1.000 - ETA: 12s - loss: 0.0516 - accuracy: 0.994 - ETA: 11s - loss: 0.0618 - accuracy: 0.991 - ETA: 11s - loss: 0.0663 - accuracy: 0.988 - ETA: 11s - loss: 0.0676 - accuracy: 0.986 - ETA: 10s - loss: 0.0633 - accuracy: 0.987 - ETA: 10s - loss: 0.0604 - accuracy: 0.988 - ETA: 10s - loss: 0.0623 - accuracy: 0.987 - ETA: 10s - loss: 0.0638 - accuracy: 0.985 - ETA: 9s - loss: 0.0678 - accuracy: 0.984 - ETA: 9s - loss: 0.0682 - accuracy: 0.98 - ETA: 9s - loss: 0.0680 - accuracy: 0.98 - ETA: 9s - loss: 0.0696 - accuracy: 0.98 - ETA: 9s - loss: 0.0685 - accuracy: 0.98 - ETA: 8s - loss: 0.0673 - accuracy: 0.98 - ETA: 8s - loss: 0.0663 - accuracy: 0.98 - ETA: 8s - loss: 0.0662 - accuracy: 0.98 - ETA: 7s - loss: 0.0660 - accuracy: 0.98 - ETA: 7s 

63/63 [==============================] - ETA: 19s - loss: 0.0791 - accuracy: 1.000 - ETA: 15s - loss: 0.0771 - accuracy: 1.000 - ETA: 13s - loss: 0.0689 - accuracy: 1.000 - ETA: 13s - loss: 0.0606 - accuracy: 1.000 - ETA: 12s - loss: 0.0576 - accuracy: 1.000 - ETA: 12s - loss: 0.0589 - accuracy: 0.994 - ETA: 11s - loss: 0.0647 - accuracy: 0.986 - ETA: 11s - loss: 0.0618 - accuracy: 0.984 - ETA: 11s - loss: 0.0580 - accuracy: 0.986 - ETA: 10s - loss: 0.0555 - accuracy: 0.987 - ETA: 10s - loss: 0.0530 - accuracy: 0.988 - ETA: 10s - loss: 0.0509 - accuracy: 0.989 - ETA: 10s - loss: 0.0502 - accuracy: 0.990 - ETA: 9s - loss: 0.0490 - accuracy: 0.991 - ETA: 9s - loss: 0.0489 - accuracy: 0.99 - ETA: 9s - loss: 0.0484 - accuracy: 0.99 - ETA: 9s - loss: 0.0475 - accuracy: 0.99 - ETA: 8s - loss: 0.0465 - accuracy: 0.99 - ETA: 8s - loss: 0.0503 - accuracy: 0.99 - ETA: 8s - loss: 0.0498 - accuracy: 0.99 - ETA: 8s - loss: 0.0496 - accuracy: 0.98 - ETA: 8s - loss: 0.0511 - accuracy: 0.99 - ETA: 7s 

63/63 [==============================] - ETA: 20s - loss: 0.0412 - accuracy: 1.000 - ETA: 16s - loss: 0.0490 - accuracy: 1.000 - ETA: 14s - loss: 0.0384 - accuracy: 1.000 - ETA: 13s - loss: 0.0308 - accuracy: 1.000 - ETA: 13s - loss: 0.0282 - accuracy: 1.000 - ETA: 12s - loss: 0.0271 - accuracy: 1.000 - ETA: 12s - loss: 0.0267 - accuracy: 1.000 - ETA: 12s - loss: 0.0261 - accuracy: 1.000 - ETA: 11s - loss: 0.0255 - accuracy: 1.000 - ETA: 11s - loss: 0.0253 - accuracy: 1.000 - ETA: 11s - loss: 0.0249 - accuracy: 1.000 - ETA: 10s - loss: 0.0242 - accuracy: 1.000 - ETA: 10s - loss: 0.0240 - accuracy: 1.000 - ETA: 10s - loss: 0.0239 - accuracy: 1.000 - ETA: 10s - loss: 0.0238 - accuracy: 1.000 - ETA: 9s - loss: 0.0245 - accuracy: 1.000 - ETA: 9s - loss: 0.0249 - accuracy: 1.00 - ETA: 9s - loss: 0.0244 - accuracy: 1.00 - ETA: 9s - loss: 0.0250 - accuracy: 1.00 - ETA: 8s - loss: 0.0248 - accuracy: 1.00 - ETA: 8s - loss: 0.0251 - accuracy: 1.00 - ETA: 8s - loss: 0.0251 - accuracy: 1.00 - ETA:

63/63 [==============================] - ETA: 19s - loss: 0.0173 - accuracy: 1.000 - ETA: 15s - loss: 0.0125 - accuracy: 1.000 - ETA: 14s - loss: 0.0159 - accuracy: 1.000 - ETA: 13s - loss: 0.0183 - accuracy: 1.000 - ETA: 13s - loss: 0.0192 - accuracy: 1.000 - ETA: 12s - loss: 0.0206 - accuracy: 1.000 - ETA: 12s - loss: 0.0199 - accuracy: 1.000 - ETA: 11s - loss: 0.0250 - accuracy: 1.000 - ETA: 11s - loss: 0.0239 - accuracy: 1.000 - ETA: 11s - loss: 0.0233 - accuracy: 1.000 - ETA: 11s - loss: 0.0243 - accuracy: 1.000 - ETA: 10s - loss: 0.0235 - accuracy: 1.000 - ETA: 10s - loss: 0.0232 - accuracy: 1.000 - ETA: 10s - loss: 0.0224 - accuracy: 1.000 - ETA: 9s - loss: 0.0219 - accuracy: 1.000 - ETA: 9s - loss: 0.0222 - accuracy: 1.00 - ETA: 9s - loss: 0.0217 - accuracy: 1.00 - ETA: 9s - loss: 0.0208 - accuracy: 1.00 - ETA: 8s - loss: 0.0207 - accuracy: 1.00 - ETA: 8s - loss: 0.0201 - accuracy: 1.00 - ETA: 8s - loss: 0.0210 - accuracy: 1.00 - ETA: 8s - loss: 0.0207 - accuracy: 1.00 - ETA: 8

63/63 [==============================] - ETA: 29s - loss: 0.0044 - accuracy: 1.000 - ETA: 22s - loss: 0.0091 - accuracy: 1.000 - ETA: 19s - loss: 0.0115 - accuracy: 1.000 - ETA: 18s - loss: 0.0099 - accuracy: 1.000 - ETA: 16s - loss: 0.0095 - accuracy: 1.000 - ETA: 16s - loss: 0.0096 - accuracy: 1.000 - ETA: 15s - loss: 0.0100 - accuracy: 1.000 - ETA: 14s - loss: 0.0101 - accuracy: 1.000 - ETA: 14s - loss: 0.0104 - accuracy: 1.000 - ETA: 13s - loss: 0.0100 - accuracy: 1.000 - ETA: 13s - loss: 0.0094 - accuracy: 1.000 - ETA: 12s - loss: 0.0091 - accuracy: 1.000 - ETA: 12s - loss: 0.0109 - accuracy: 1.000 - ETA: 12s - loss: 0.0109 - accuracy: 1.000 - ETA: 11s - loss: 0.0108 - accuracy: 1.000 - ETA: 11s - loss: 0.0106 - accuracy: 1.000 - ETA: 11s - loss: 0.0103 - accuracy: 1.000 - ETA: 10s - loss: 0.0104 - accuracy: 1.000 - ETA: 10s - loss: 0.0105 - accuracy: 1.000 - ETA: 10s - loss: 0.0108 - accuracy: 1.000 - ETA: 9s - loss: 0.0108 - accuracy: 1.000 - ETA: 9s - loss: 0.0106 - accuracy: 1

In [65]:
loss, accuracy = model.evaluate_generator(validation_generator)

  ...
    to  
  ['...']


In [67]:
print('Loss: {}, Accuracy: {}'.format(loss, accuracy))

Loss: 2.3717147558927536, Accuracy: 0.5


 # Fine Tuning

In [68]:
base_model.trainable = True

In [69]:
print('Number of layers in the base_model : {}'.format(len(base_model.layers)))

Number of layers in the base_model : 155


In [70]:
fine_tune_at = 100

In [71]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [72]:
base_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [74]:
model.fit_generator(train_generator, epochs=5, validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 63 steps, validate for 32 steps
Epoch 1/5
63/63 [==============================] - ETA: 19s - loss: 0.0114 - accuracy: 1.000 - ETA: 15s - loss: 0.0096 - accuracy: 1.000 - ETA: 14s - loss: 0.0077 - accuracy: 1.000 - ETA: 13s - loss: 0.0103 - accuracy: 1.000 - ETA: 12s - loss: 0.0096 - accuracy: 1.000 - ETA: 12s - loss: 0.0090 - accuracy: 1.000 - ETA: 11s - loss: 0.0080 - accuracy: 1.000 - ETA: 11s - loss: 0.0084 - accuracy: 1.000 - ETA: 11s - loss: 0.0081 - accuracy: 1.000 - ETA: 10s - loss: 0.0082 - accuracy: 1.000 - ETA: 10s - loss: 0.0077 - accuracy: 1.000 - ETA: 10s - loss: 0.0078 - accuracy: 1.000 - ETA: 10s - loss: 0.0082 - accuracy: 1.000 - ETA: 9s - loss: 0.0090 - accuracy: 1.000 - ETA: 9s - loss: 0.0090 - accuracy: 1.00 - ETA: 9s - loss: 0.0089 - accuracy: 1.00 - ETA: 9s - loss: 0.0101 - accuracy: 1.00 - ETA: 9s - loss: 0.0108 - accuracy: 1.00 - ETA: 8s - loss: 0.0106 - accuracy: 1.00 - ETA: 8s - loss: 0.0106 - accurac

63/63 [==============================] - ETA: 18s - loss: 0.0034 - accuracy: 1.000 - ETA: 15s - loss: 0.0044 - accuracy: 1.000 - ETA: 14s - loss: 0.0044 - accuracy: 1.000 - ETA: 11s - loss: 0.0039 - accuracy: 1.000 - ETA: 11s - loss: 0.0047 - accuracy: 1.000 - ETA: 11s - loss: 0.0042 - accuracy: 1.000 - ETA: 11s - loss: 0.0041 - accuracy: 1.000 - ETA: 11s - loss: 0.0040 - accuracy: 1.000 - ETA: 10s - loss: 0.0040 - accuracy: 1.000 - ETA: 10s - loss: 0.0040 - accuracy: 1.000 - ETA: 10s - loss: 0.0040 - accuracy: 1.000 - ETA: 10s - loss: 0.0043 - accuracy: 1.000 - ETA: 9s - loss: 0.0043 - accuracy: 1.000 - ETA: 9s - loss: 0.0042 - accuracy: 1.00 - ETA: 9s - loss: 0.0050 - accuracy: 1.00 - ETA: 9s - loss: 0.0049 - accuracy: 1.00 - ETA: 9s - loss: 0.0057 - accuracy: 1.00 - ETA: 8s - loss: 0.0056 - accuracy: 1.00 - ETA: 8s - loss: 0.0056 - accuracy: 1.00 - ETA: 8s - loss: 0.0058 - accuracy: 1.00 - ETA: 8s - loss: 0.0059 - accuracy: 1.00 - ETA: 7s - loss: 0.0058 - accuracy: 1.00 - ETA: 7s - 

63/63 [==============================] - ETA: 20s - loss: 0.0019 - accuracy: 1.000 - ETA: 13s - loss: 0.0042 - accuracy: 1.000 - ETA: 12s - loss: 0.0036 - accuracy: 1.000 - ETA: 12s - loss: 0.0036 - accuracy: 1.000 - ETA: 11s - loss: 0.0037 - accuracy: 1.000 - ETA: 11s - loss: 0.0033 - accuracy: 1.000 - ETA: 11s - loss: 0.0031 - accuracy: 1.000 - ETA: 11s - loss: 0.0029 - accuracy: 1.000 - ETA: 11s - loss: 0.0029 - accuracy: 1.000 - ETA: 10s - loss: 0.0029 - accuracy: 1.000 - ETA: 10s - loss: 0.0031 - accuracy: 1.000 - ETA: 10s - loss: 0.0029 - accuracy: 1.000 - ETA: 10s - loss: 0.0031 - accuracy: 1.000 - ETA: 9s - loss: 0.0030 - accuracy: 1.000 - ETA: 9s - loss: 0.0031 - accuracy: 1.00 - ETA: 9s - loss: 0.0030 - accuracy: 1.00 - ETA: 9s - loss: 0.0030 - accuracy: 1.00 - ETA: 9s - loss: 0.0029 - accuracy: 1.00 - ETA: 8s - loss: 0.0029 - accuracy: 1.00 - ETA: 8s - loss: 0.0029 - accuracy: 1.00 - ETA: 8s - loss: 0.0032 - accuracy: 1.00 - ETA: 8s - loss: 0.0031 - accuracy: 1.00 - ETA: 7s 

In [75]:
loss, accuracy = model.evaluate_generator(validation_generator)

  ...
    to  
  ['...']


In [76]:
print('Loss: {}, Accuracy: {}'.format(loss, accuracy))

Loss: 3.5320415645837784, Accuracy: 0.5
